# <font color='blue'>Data Science Academy - Formação Cientista de Dados</font>
# <font color='blue'>Autor: Evandro Eulálio Cleto</font>

## <font color='blue'>Data Início: 07/06/2023</font>
## <font color='blue'>Data Finalização: 13/06/2023</font>


![title](imagens/Projeto_imagem.png)

## <font color='blue'>Objetivo deste projeto:</font>
### <font color='blue'>Através da análise de Tweets sobre o ChatGPT foi construído um processo de análise que permite identificar o sentimento que predomina, especialmente no Twitter, sobre o ChatGPT.</font>

Resumo do Projeto: Criar um projeto de previsão de sentimentos sobre ChatGPT atráves de Tweets on-line usando Machine Learning.
Os sentimentos serão previstos como positivo, negativo ou neutro.

Acesse http://localhost:4040 para acompanhar a execução dos jobs

## Spark Streaming - Twitter

In [ ]:
# Instalação de pacotes necessários para o projeto
#!pip install requests_oauthlib
#!pip install twython
#!pip install nltk
#!pip install emoji
#!pip install imblearn


In [ ]:
# https://pypi.org/project/findspark/
!pip install -q findspark

In [ ]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [ ]:
# Módulos usados
from pyspark.streaming import StreamingContext
#from pyspark.streaming.twitter import TwitterUtils
from pyspark import SparkContext
from pyspark.sql import SparkSession
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter
import re
import requests
import time
import string
import ast
import json
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
# Baixa Stopwords do pacote NLTK
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
nltk.download("stopwords")

In [ ]:
# Importa o arquivo CSV como DataFrame do pandas
# Esse é um dataset com 219294 registros de tweets do Chat GPT rotulado com sentimentos positivo, negativo que 
# será utilizado para treinamento do NaiveBayesClassifier.
# O dataset foi obtido em:  https://www.kaggle.com/datasets/charunisa/chatgpt-sentiment-analysis
df = pd.read_csv("dados/ChatGPT_sentiment_orig.csv",sep=",")

In [ ]:
df.head(10)

In [ ]:
# Verificando o tipo do objeto
type(df)

In [ ]:
# Verificando o shape dos dados
df.shape

In [ ]:
# Remove a coluna 'code' pois não tem relevância para o projeto
df = df.drop('code', axis=1)

In [ ]:
df.head(10)

In [ ]:
# Altera posição da coluna 'labels' para a posição 0
cols = df.columns.tolist()
cols = ['labels'] + cols[:cols.index('labels')] + cols[cols.index('labels')+1:]
df = df[cols]

In [ ]:
df.head(10)

In [ ]:
# Remove vírgulas, exceto as do final da linha, da coluna 'tweets' evitar problemas na função que remove pontuação
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'(?<!\n),', '', x))

In [ ]:
df.head(10)

In [ ]:
# Mapear as classes para valores numéricos
label_mapping = {'bad': 0, 'good': 1, 'neutral': 2}
df['labels'] = df['labels'].map(label_mapping)

In [ ]:
df.head(10)

In [ ]:
# Remover emojis e caracteres especiais da coluna 'tweets'
df['tweets'] = df['tweets'].apply(lambda x: re.sub(r'[^\w\s,]', '', x))

In [ ]:
df.head(10)

In [ ]:
### Função para retirar o conteúdo  HTTP da coluna  tweets
def remove_urls(text):
    # Padrão para identificar URLs com "http" seguido por não espaços
    pattern = r"http\S+"
    # Substituir o padrão pela string vazia
    clean_text = re.sub(pattern, "", text)
    return clean_text

In [ ]:
# Aplicar a função à coluna "tweets" do DataFrame
df["tweets"] = df["tweets"].apply(remove_urls)

In [ ]:
type(df)

### Tratamento do dataset para filtro dos tweets no idíoma inglês

In [ ]:
# Obtém as stopwords em todos os idiomas
dicionario_stopwords = {lang: set(nltk.corpus.stopwords.words(lang)) for lang in nltk.corpus.stopwords.fileids()}

dicionario_stopwords

In [ ]:
# Função para detectar o idioma predominante com base nas stopswords
def descobre_idioma(text):
    
    #Aplica tokenização considerando pontuação
    palavras = set(nltk.wordpunct_tokenize(text.lower()))
    
    # Conta o total de palavras tokenizadas considerando o dicionario de stopwords
    lang = max(((lang, len(palavras & stopwords)) for lang, stopwords in dicionario_stopwords.items()), key = lambda x: x[1])[0]
    
    # Verifica se o idioma é Inglês
    if lang == 'english':
        return True
    else:
        return False    

In [ ]:
# Filtra somente os comentários em Inglês
df_ingles = df[df['tweets'].apply(descobre_idioma)]

In [ ]:
df_ingles.head(10)

In [ ]:
# Verificando o shape dos dados
df_ingles.shape

In [ ]:
# Função para remover caracteres não latinos pois existem tweets que apesar de estar em Ingles possuem caratcteres 
# de linguas como Chinês, Japonês, Russo, etc
def remove_non_latin_chars(text):
    # Padrão para identificar caracteres não latinos
    pattern = r'[^\x00-\x7F]+'
    # Substituir o padrão pela string vazia
    clean_text = re.sub(pattern, '', text)
    return clean_text

In [ ]:
# Aplicar a função à coluna "tweets" do DataFrame
df_ingles["tweets"] = df_ingles["tweets"].apply(remove_non_latin_chars)

In [ ]:
# Verificado que exisate linhas com conteúdo ChatGPT que são propagandas de site de pornografia e serão removidos
# Função para remover linhas com a palavra "nMaandamano twitterfiles"
def remove_lines_with_keywords(text):
    keywords = ["nMaandamano twitterfiles", "AryanKhan ChatGPT","Products launched"]
    for keyword in keywords:
        if keyword in text:
            return ""
    return text

In [ ]:
# Aplicar a função à coluna "tweets" do DataFrame
df_ingles["tweets"] = df_ingles["tweets"].apply(remove_lines_with_keywords)

In [ ]:
# Remover linhas vazias resultantes
df_ingles = df_ingles[df_ingles["tweets"] != ""]


In [ ]:
df_ingles.head(10)

In [ ]:
# Resetar o índice do DataFrame
df_ingles.reset_index(drop=True, inplace=True)

In [ ]:
df_ingles.head(10)

In [ ]:
# Verificando o shape dos dados
df_ingles.shape

In [ ]:
# Função para su bstituir "nn", "\n" ou "n" seguido de número no início ou no fim de uma palavra pela string vazia
def remove_special_chars(text):
    # Substituir "nn", "\n", "n" seguido de número ou "nn" seguido de número no início ou no fim de uma palavra pela string vazia
    cleaned_text = re.sub(r'\b(nn|\n|n\d+|nn\d+)\b', '', text)
    return cleaned_text

In [ ]:
# Aplicar a função à coluna "tweets" do DataFrame
df_ingles["tweets"] = df_ingles["tweets"].apply(remove_special_chars)

In [ ]:
# Verificando o shape dos dados
df_ingles.shape

In [ ]:
# Removendo duplicidade
df_ingles.drop_duplicates(inplace=True)

In [ ]:
# Verificando o shape dos dados
df_ingles.shape

In [ ]:
# Salva o dataframe tratado em csv
df_ingles.to_csv('dados/ChatGPT_sentiment_limpo.csv', index=False)

In [ ]:
# Frequência de update
INTERVALO_BATCH = 5

In [ ]:
# Cria o Spark Context
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [ ]:
## Criando o StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

## Treinando o Classificador de Análise de Sentimento

O dataset limpo e filtrado pelo idioma Inglês contém 44785 tweets classificados e cada linha é marcada como: 

### 0 para o sentimento negativo 
### 1 para o sentimento positivo 
### 2 para o sentimento neutro 

In [ ]:
## Lendo o arquivo texto e criando um RDD em memória com Spark
arquivo = sc.textFile("dados/ChatGPT_sentiment_limpo.csv")

In [ ]:
arquivo.collect()

In [ ]:
##Removendo o cabeçalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line!=header)

In [ ]:
dataset.collect()

In [ ]:
type(dataset)

In [ ]:
## Essa função separa as colunas em cada linha, cria uma tupla e remove a pontuação
def get_row(line):
    row = line.split(',')
    sentimento = row[0]
    tweet = row[1].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return(tweet_lower,sentimento)

In [ ]:
#Aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
#Cria um objeto SentimentAnalyser
sentiment_analyser = SentimentAnalyzer()

In [ ]:
# Obtem a lista de stopwords
stopwords_all = []
for word in stopwords.words('english'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

In [ ]:
#Obtem 31.350(70%) tweets do dataset de treino e retorna todas as palavras que não são Stpwords
dataset_treino_amostra = dataset_treino.take(4200)

In [ ]:
dataset_treino_amostra

In [ ]:
all_words_neg = sentiment_analyser.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
#Cria um unigram(n-grama) e extrai as features
unigram_feats = sentiment_analyser.unigram_word_feats(all_words_neg_nostops, top_n=200)
sentiment_analyser.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyser.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

In [ ]:
print(training_set)

In [ ]:
# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyser.train(trainer, training_set)

In [ ]:
# Testa o classificador em algumas sentenças
test_sentence1 = [(['model', 'is', 'people', 'bad'], '')]
test_sentence2 = [(['learning', 'day', 'bit', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'results', 'awesome'], '')]
test_set = sentiment_analyser.apply_features(test_sentence1)
test_set2 = sentiment_analyser.apply_features(test_sentence2)
test_set3 = sentiment_analyser.apply_features(test_sentence3)

In [ ]:
test_set

In [ ]:
test_set2

In [ ]:
test_set3

In [ ]:
#Configurando o Stream
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default=rdd)

In [ ]:
#Total de Tweets por update
NUM_TWEETS = 500

In [ ]:
type(stream)

In [ ]:
# Essa função conecta ao Twitter e retorna um número específico de Tweets (NUM_TWEETS)
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
   #response = requests.get(filter_url, auth = auth, stream = True)
  response = requests.get(filter_url, auth = auth, headers=auth_header, stream=True, params=query_params)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
stream

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
  sentence = [(tweet, '')]
  test_set = sentiment_analyzer.apply_features(sentence)
  print(tweet, classifier.classify(test_set[0][0]))
  return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
# Essa função retorna o texto do Twitter
def get_tweet_text(rdd):
  for line in rdd:
    tweet = line.strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
      tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [ ]:
# Cria uma lista vazia para os resultados
resultados = []

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
  global resultados
  pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
  counts = pairs.reduceByKey(add)
  output = []
  for count in counts.collect():
    output.append(count)
  result = [time.strftime("%I:%M:%S"), output]
  resultados.append(result)
  print(result)

In [ ]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
# Start streaming
ssc.start()
# ssc.awaitTermination()

In [ ]:
cont = True
while cont:
  if len(resultados) > 5:
    cont = False

In [ ]:
# Grava os resultados
rdd_save = '/dados/r'+time.strftime("%I%M%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

In [ ]:
# Visualiza os resultados
resultados_rdd.collect()

In [ ]:
# Finaliza o streaming
ssc.stop()